Projeto Final: Análise Preditiva de Desfechos em Casos de SRAG

Disciplina: Introdução à Ciência de Dados - IFB Campus Brasília Estudantes:

  - Davi Campos Parente

  - Marques Herminio

Data da Entrega: 26/11/2025

- Definição do Subproblema
  - Título do Subproblema: Predição de Óbito em Pacientes Hospitalizados por SRAG.

# Objetivo: Desenvolver um modelo de classificação supervisionada utilizando o algoritmo Árvore de Decisão para prever o desfecho clínico (Cura ou Óbito) de pacientes hospitalizados com Síndrome Respiratória Aguda Grave (SRAG).

Escolheu-se a Árvore de Decisão devido à sua alta interpretabilidade, permitindo não apenas a predição, mas também a visualização clara das regras de decisão e dos fatores de risco mais determinantes.

2. Período: 2019 a 2025 (Abrangendo o período pré-pandêmico, pandêmico e pós-pandêmico da COVID-19).

Fonte: Dados abertos do Ministério da Saúde (OpenDatasus).

3. Seleção de Variáveis Relevantes
Com base no Dicionário de Dados do SIVEP-Gripe, foram selecionadas as seguintes variáveis para a modelagem:

Variável Alvo (Target):


EVOLUCAO: Desfecho do caso. Focaremos nas categorias 1-Cura e 2-Óbito. (Casos com evolução 3-Óbito por outras causas ou 9-Ignorado serão filtrados).

Variáveis Preditoras (Features):

Demográficas:


NU_IDADE_N: Idade do paciente.

CS_SEXO: Sexo biológico.

CS_RACA: Raça/Cor.

Sintomas Clínicos (Sinais de Gravidade):

FEBRE, TOSSE, GARGANTA, DISPNEIA (Dificuldade para respirar), DESC_RESP (Desconforto respiratório), SATURACAO (Saturação de O2 < 95%).

Fatores de Risco e Comorbidades:


FATOR_RISC: Presença de algum fator de risco (Sim/Não).


CARDIOPATI (Doença Cardiovascular), DIABETES, OBESIDADE, ASMA, IMUNODEPRE (Imunodepressão).

Gestão Clínica:


UTI: Se houve internação em UTI.


SUPORT_VEN: Uso de suporte ventilatório (invasivo/não invasivo).


VACINA_COV: Se recebeu vacina COVID-19.

4. Identificação Inicial de Problemas nos Dados
Na análise preliminar da base e do dicionário, identificamos os seguintes desafios que serão tratados na etapa de pré-processamento:

Valores Ausentes e "Ignorados":

Grande parte das variáveis categóricas (como sintomas e raça) utiliza o código 9 para representar "Ignorado". Estes precisarão ser tratados (convertidos para NaN ou uma categoria específica) para não distorcer o modelo.



Inconsistências de Datas:

Há regras de validação lógica, como DT_INTERNA (Data de Internação) não podendo ser anterior à DT_SIN_PRI (Data dos Primeiros Sintomas). Registros que violem essa lógica temporal precisarão ser removidos.

Tipagem de Dados:

As colunas de datas geralmente são carregadas como strings (objetos) e precisarão ser convertidas para o tipo datetime.

Variáveis numéricas categóricas (1, 2, 9) devem ser tratadas como categorias, não como números contínuos.

Balanceamento de Classes:

É necessário verificar a proporção entre casos de Cura e Óbito. Se houver desbalanceamento severo, técnicas de balanceamento podem ser necessárias para evitar viés no modelo.

In [ ]:
import pandas as pd
import numpy as np

# =============================================================================
# 1. COLETA E CARREGAMENTO DOS DADOS
# =============================================================================
# Carregamento da base de dados SRAG (2019-2025).
# Utiliza-se separador ';' e encoding 'latin1' conforme padrão do DATASUS.
# Nota: O arquivo deve estar no mesmo diretório do notebook ou o caminho deve ser ajustado.
try:
    df = pd.read_csv('influd21_limpo-final.csv', sep=';', encoding='latin1', low_memory=False)
    print(f"Base de dados carregada com sucesso. Dimensões iniciais: {df.shape}")
except FileNotFoundError:
    print("Erro: Arquivo não encontrado. Verifique o caminho e o nome do arquivo.")

# =============================================================================
# 2. SELEÇÃO DE VARIÁVEIS (FEATURE SELECTION)
# =============================================================================
# Definição das variáveis relevantes para o subproblema de "Predição de Óbito",
# baseadas no Dicionário de Dados SIVEP-Gripe.

colunas_interesse = [
    # Variável Alvo (Target)
    'EVOLUCAO',     # 1-Cura, 2-Óbito

    # Dados Demográficos
    'NU_IDADE_N',   # Idade
    'CS_SEXO',      # Sexo
    'CS_RACA',      # Raça/Cor

    # Sinais e Sintomas (Variáveis Clínicas)
    'FEBRE',
    'TOSSE',
    'GARGANTA',
    'DISPNEIA',
    'SATURACAO',
    'DESC_RESP',

    # Fatores de Risco e Comorbidades
    'FATOR_RISC',   # Presença de fator de risco (Sim/Não)
    'CARDIOPATI',
    'DIABETES',
    'OBESIDADE',
    'ASMA',
    'IMUNODEPRE',

    # Gestão Clínica e Intervenções
    'UTI',          # Internação em UTI
    'SUPORT_VEN',   # Uso de suporte ventilatório
    'VACINA_COV',   # Histórico de vacinação COVID-19

    # Datas (para validação temporal e consistência)
    'DT_SIN_PRI',   # Data primeiros sintomas
    'DT_INTERNA'    # Data da internação
]

# Filtragem do DataFrame para manter apenas as colunas existentes na base carregada
cols_existentes = [c for c in colunas_interesse if c in df.columns]
df_selecionado = df[cols_existentes].copy()

print(f"Seleção de variáveis concluída. Colunas mantidas: {df_selecionado.shape[1]}")

# =============================================================================
# 3. PRÉ-PROCESSAMENTO E LIMPEZA DE DADOS
# =============================================================================

# 3.1. Filtragem da Variável Alvo
# Manter apenas casos com desfecho definido: 1 (Cura) e 2 (Óbito).
# Casos com '3' (Óbito outras causas) ou '9' (Ignorado) são removidos da análise.
if 'EVOLUCAO' in df_selecionado.columns:
    df_limpo = df_selecionado[df_selecionado['EVOLUCAO'].isin([1, 2])].copy()
else:
    df_limpo = df_selecionado.copy()

# 3.2. Tratamento de Valores 'Ignorados' (Código 9)
# Conversão do código '9' (Ignorado) para NaN (Not a Number) nas variáveis categóricas
# e de sintomas, permitindo a correta identificação de dados ausentes.
cols_numericas_cat = ['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'SATURACAO',
                      'DESC_RESP', 'FATOR_RISC', 'CARDIOPATI', 'DIABETES',
                      'OBESIDADE', 'ASMA', 'IMUNODEPRE', 'UTI', 'SUPORT_VEN']

cols_para_tratamento = [c for c in cols_numericas_cat if c in df_limpo.columns]

for col in cols_para_tratamento:
    # A função replace é utilizada para padronizar a ausência de informação
    df_limpo[col] = df_limpo[col].replace(9, np.nan)

# Exibição da contagem de valores nulos após o tratamento
print("\n--- Contagem de Valores Ausentes (NaN) após tratamento do código 9 ---")
print(df_limpo[cols_para_tratamento].isnull().sum())

# 3.3. Conversão e Validação de Datas
# Conversão das colunas de data para o tipo datetime.
datas_cols = ['DT_SIN_PRI', 'DT_INTERNA']
for col in datas_cols:
    if col in df_limpo.columns:
        df_limpo[col] = pd.to_datetime(df_limpo[col], dayfirst=True, errors='coerce')

# Verificação de inconsistência lógica: Data de Internação anterior à Data de Sintomas.
if 'DT_INTERNA' in df_limpo.columns and 'DT_SIN_PRI' in df_limpo.columns:
    inconsistentes = df_limpo[df_limpo['DT_INTERNA'] < df_limpo['DT_SIN_PRI']]
    qtd_inconsistentes = len(inconsistentes)
    print(f"\nRegistros com inconsistência temporal (Internação < Sintomas): {qtd_inconsistentes}")

# =============================================================================
# 4. VISUALIZAÇÃO DA AMOSTRA FINAL
# =============================================================================
print("\n--- Amostra dos dados processados (Primeiros 5 registros) ---")
display(df_limpo.head())
print(f"\nDimensões finais do dataset para modelagem: {df_limpo.shape}")

Erro: Arquivo não encontrado. Verifique o caminho e o nome do arquivo.


NameError: name 'df' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Entrega_Final_ICDD/Dados'


In [ ]:
import os
os.listdir(path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Entrega_Final_ICDD/Dados'

In [ ]:
import os
os.listdir('/content/drive/Dados')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Dados'

In [ ]:
!ls /content/drive



MyDrive


In [ ]:
!ls /content/drive/MyDrive



 analise_estudos.gdoc
'Apresentação sem título.gslides'
'atividade PDDS.gdoc'
 attachment.pdf
 BRASÍLIA-SLIDE-16x9-WIDE.gslides
 Certificado_Nacional_de_Covid-19.pdf
'CNH Digital.pdf'
'Colab Notebooks'
 concursos.ods
'Cópia de TCC - Leandro Campos Sousa e Kennedy Viana Aguiar.pdf'
 Cronograma.gsheet
 Dados
'docs (1).pdf'
 docs.pdf
'Documento sem título (10).gdoc'
'Documento sem título (1).gdoc'
'Documento sem título (2).gdoc'
'Documento sem título (3).gdoc'
'Documento sem título (4).gdoc'
'Documento sem título (5).gdoc'
'Documento sem título (6).gdoc'
'Documento sem título (7).gdoc'
'Documento sem título (8).gdoc'
'Documento sem título (9).gdoc'
'Documento sem título.gdoc'
 escolaridade.pdf
'estou treinando 5x na semana, sendo:'$'\n\n''treino 1: P....gdoc'
'eu alterei o valor da semana 7 para 5:00 e no gra....gsheet'
 foto.jpg
'Google Earth'
'Historico-Graducao-Marques Herminio de Sousa Silva.pdf'
'IFB - FIC'
'me ajuda a montar esse grafico do progresso passo....gsheet'

In [ ]:
import os
os.listdir('/content/drive/MyDrive/Dados')


['Untitled',
 'INFLUD24-26-06-2025.parquet',
 'INFLUD23-26-06-2025.parquet',
 'INFLUD22-26-06-2025.parquet',
 'INFLUD21-26-06-2025.parquet',
 'INFLUD20-26-06-2025.parquet',
 'INFLUD19-26-06-2025.parquet']

In [ ]:
import pandas as pd
import os

path = '/content/drive/MyDrive/Dados'

arquivos = [f for f in os.listdir(path) if f.endswith('.parquet')]

print(f'Arquivos Parquet encontrados: {arquivos}')

dfs = []
for f in arquivos:
    print(f'Lendo {f}')
    df_temp = pd.read_parquet(os.path.join(path, f))
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

print(f'\nBase SRAG carregada com sucesso.')
print(f'Dimensões: {df.shape}')


Arquivos Parquet encontrados: ['INFLUD24-26-06-2025.parquet', 'INFLUD23-26-06-2025.parquet', 'INFLUD22-26-06-2025.parquet', 'INFLUD21-26-06-2025.parquet', 'INFLUD20-26-06-2025.parquet', 'INFLUD19-26-06-2025.parquet']
Lendo INFLUD24-26-06-2025.parquet
Lendo INFLUD23-26-06-2025.parquet
Lendo INFLUD22-26-06-2025.parquet
Lendo INFLUD21-26-06-2025.parquet
Lendo INFLUD20-26-06-2025.parquet
Lendo INFLUD19-26-06-2025.parquet


In [ ]:
import pandas as pd
import os

path = '/content/drive/MyDrive/Dados'

arquivos = sorted([f for f in os.listdir(path) if f.endswith('.parquet')])

dfs = []
for f in arquivos:
    print(f'Lendo {f}')
    df_temp = pd.read_parquet(os.path.join(path, f))

    # Amostra de 10% de cada ano
    df_temp = df_temp.sample(frac=0.1, random_state=42)

    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

print(df.shape)


Lendo INFLUD19-26-06-2025.parquet
Lendo INFLUD20-26-06-2025.parquet
Lendo INFLUD21-26-06-2025.parquet
Lendo INFLUD22-26-06-2025.parquet
Lendo INFLUD23-26-06-2025.parquet
Lendo INFLUD24-26-06-2025.parquet
(410956, 194)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive


MyDrive


In [ ]:
import pandas as pd
import os

arquivos = [f for f in os.listdir(path) if f.endswith('.parquet')]

dfs = []
for f in arquivos:
    print(f"Lendo arquivo: {f}")
    df_temp = pd.read_parquet(os.path.join(path, f))
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

print(f"Base carregada com sucesso. Dimensões iniciais: {df.shape}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/Entrega_Final_ICDD/Dados'

# Nova seção

In [ ]:
import os

pasta = "/content/drive/MyDrive/Entrega_Final_ICDD/Dados"
os.listdir(pasta)

['INFLUD24-26-06-2025.parquet',
 'INFLUD23-26-06-2025.parquet',
 'INFLUD22-26-06-2025.parquet',
 'INFLUD21-26-06-2025.parquet',
 'INFLUD20-26-06-2025.parquet',
 'INFLUD19-26-06-2025.parquet']

In [ ]:
import glob
import pandas as pd

arquivo_parquet = glob.glob("/content/drive/MyDrive/Entrega_Final_ICDD/Dados/*.parquet")

dfs = []

for arquivo in arquivo_parquet:
    print("Lendo:",arquivo)
    df = pd.read_parquet(arquivo)
    dfs.append(df)


In [ ]:
!pip install polars
import polars as pl
import glob
import os

pasta = "/content/drive/MyDrive/Entrega_Final_ICDD/Dados/*.parquet"
arquivos = glob.glob(os.path.join(pasta, "*.parquet"))

dfs = []

for arquivo in arquivos:
  print("Lendo:", arquivo)
  df = pl.read_parquet(arquivo)
  dfs.append(df)

dados = pl.concat(dfs)

ValueError: cannot concat empty list